In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
df = pd.read_csv('Model 1 dataset.csv')

In [4]:
features = [
    'current_space_utilization', 'historical_space_utilization',
    'average_weekly_space_utilization', 'total_inventory_items',
    'incoming_shipments', 'outgoing_shipments', 'inventory_turnover_rate',
    'product_categories', 'bulk_items_count', 'perishable_items_count',
    'seasonal_products_count', 'staff_count', 'operational_hours',
    'equipment_usage', 'market_trends', 'promotional_events',
    'supplier_delays', 'economic_indicators', 'special_events',
    'anomalies', 'rental_cost_per_sq_ft', 'space_savings',
    'transportation_lead_time', 'delivery_frequency',
    'historical_demand'
]

In [5]:
target = 'forecasted_demand'

In [6]:
# Prepare the data
X = df[features].values
y = df[target].values

In [7]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Reshape input to be 3D [samples, timesteps, features]
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

C:\Users\yojee\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [12]:
# Fit the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [13]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2,
                    callbacks=[early_stopping], verbose=1)

Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 2347748096.0000 - mae: 48113.4258 - val_loss: 2355600128.0000 - val_mae: 48183.9648
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2353981696.0000 - mae: 48175.9258 - val_loss: 2346986240.0000 - val_mae: 48094.4883
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2332351488.0000 - mae: 47945.1406 - val_loss: 2336631808.0000 - val_mae: 47986.7266
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2319678720.0000 - mae: 47820.7930 - val_loss: 2323725824.0000 - val_mae: 47852.0586
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2310683136.0000 - mae: 47719.6719 - val_loss: 2308165632.0000 - val_mae: 47689.1914
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2282878208.0000 - mae: 47411.8320 - val_loss: 2290006016.0000 - val_mae: 47498.4180
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2239154944.0000 - mae: 46964.4961 - val_loss: 2269278208.0000 - val_mae: 47279.726

In [14]:
# Evaluate the model on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 32571816.0000 - mae: 4939.2827
Test Loss: 33104166.0, Test MAE: 4977.0478515625


In [15]:
# Calculate accuracy
predictions = model.predict(X_test)
accuracy = 1 - (np.abs(predictions.flatten() - y_test) / y_test).mean()
print(f'Model Accuracy: {accuracy * 100:.2f}%')

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Model Accuracy: 89.41%
